In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import string
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import matplotlib.pyplot as plt
plt.style.use('seaborn')
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
from wordcloud import WordCloud
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
import nltk.data
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import word_tokenize
nltk.download('vader_lexicon')
nltk.download('punkt')
import warnings
warnings.filterwarnings("ignore")

In [21]:
cd /content/drive/MyDrive/Colab Notebooks/sNLP_project/SongRecommendationChatBot/song_dataset/csv

/content/drive/MyDrive/Colab Notebooks/sNLP_project/SongRecommendationChatBot/song_dataset/csv


Bring the csv file of Taylor Swift Songs

In [22]:
df = pd.read_csv('./TaylorSwift.csv',index_col=0)
print('Number of Taylor Swift songs:',len(df))

Number of Taylor Swift songs: 479


In [23]:
df.head()

,Artist,Title,Album,Year,Date,Lyric
0,Taylor Swift,​cardigan,folklore,2020.0,2020-07-24,vintage tee brand new phone high heels on cobb...
1,Taylor Swift,​exile,folklore,2020.0,2020-07-24,justin vernon i can see you standing honey wit...
2,Taylor Swift,Lover,Lover,2019.0,2019-08-16,we could leave the christmas lights up 'til ja...
3,Taylor Swift,​the 1,folklore,2020.0,2020-07-24,i'm doing good i'm on some new shit been sayin...
4,Taylor Swift,Look What You Made Me Do,reputation,2017.0,2017-08-25,i don't like your little games don't like your...


In [24]:
df = df.astype({"Lyric": str}, errors='raise') 

Data preprocessing (song lyrics cleaning)

In [25]:
#remove remix, live ....etc version of the songs
def cleaning(df):
    a=[]
    i=0
    df1=df
    title = df['Title']
    for t in df['Title']:
        r=Re=l=Li=c=m=V=ve=D=rs=0
        r=t.find('remix')
        Re=t.find('Remix')
        l=t.find('live')
        Li=t.find('Live')
        V=t.find('Version')
        ve=t.find('version')
        D=t.find('Demo ')
        D=t.find('Demo')
        rs=t.find('Reprise')
        c=t.find('COPY')
        m=t.find('Mix')
        if r != -1:
            a.append(t)
        elif Re != -1:
            a.append(t)
        elif l != -1:
            a.append(t)
        elif Li != -1:
            a.append(t)
        elif V != -1:
            a.append(t)
        elif ve != -1:
            a.append(t)
        elif D != -1:
            a.append(t)
        elif rs != -1:
            a.append(t)
        elif c != -1:
            a.append(t)
        elif m != -1:
            a.append(t)
    
    for t1 in df['Title']:
        for t2 in a:
            if t1 == t2:
                df1=df1.drop(i)
        i=i+1
    
    df1.dropna(subset = ["Title"], inplace=True)
    df1.dropna(subset = ["Lyric"], inplace=True)
    df1.drop_duplicates(subset ="Title",keep = False, inplace = True)
    df1.drop_duplicates(subset ="Lyric",keep = False, inplace = True) 
    
    
    return df1

In [27]:
#lyrics to word
def lyrics_to_words(document):
    stop_words = set(stopwords.words('english'))
    exclude = set(string.punctuation)
    lemma = WordNetLemmatizer()
    stopwordremoval = " ".join([i for i in document.lower().split() if i not in stop_words])
    punctuationremoval = ''.join(ch for ch in stopwordremoval if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punctuationremoval.split())
    return normalized

In [28]:
#word count in lyrics
def toword(df):
    def unique(list1):
         unique_list = []
         for x in list1:
             if x not in unique_list:
                  unique_list.append(x)
         return unique_list
    
    total_words = []
    total_word_c = []
    unique_words = []
    unique_word_c = []
    lexical_rich = []

    df = df.reset_index(drop=True)
    for word in df['cleaned'].tolist():
        word_l = word.split()
        unique_l = unique(word_l)
        total_words.append(word_l)
        total_word_c.append(len(word_l))
        unique_words.append(unique_l)
        unique_word_c.append(len(unique_l))
        lexical_rich.append(len(unique_l)/len(word_l))

    df['total_words'] = total_words
    df['total_words_c'] = total_word_c
    df['unique_words'] = unique_words
    df['unique_words_c'] = unique_word_c
    df['lexical_richment'] = lexical_rich
    return df

In [29]:
df = cleaning(df)
print('Number of songs after removing the version of songs:', len(df))
df['cleaned'] = df['Lyric'].apply(lyrics_to_words)
df = toword(df)

Number of songs after removing the version of songs: 363


In [31]:
#word count by album
df_words = df.groupby(df['Album']).sum()
df_words =df_words.reset_index()

In [33]:
#unreleased songs
df_un = df[df['Album']=='Unreleased Songs']
print('Number of unreleased songs:',df_un.shape[0])

Number of unreleased songs: 65


In [34]:
#album count
df_album_count = df.groupby(df['Album']).count()
df_album_count =df_album_count.reset_index()
print('Number of albums:',df_album_count.shape[0])

Number of albums: 38


In [35]:
#word count graph
fig = go.Figure(data=[
    go.Bar(name='Total word count', x=df_words['Album'], y=df_words['total_words_c'].tolist()),
    go.Bar(name='Unique Word Count', x=df_words['Album'], y=df['unique_words_c'].tolist()),
    
])

fig.update_layout(barmode='group',title=dict(text="Total words vs Unique words",y=0.95,x=0.45,xanchor='center',yanchor='top',font=dict(size=27)),
                  autosize=False, width=1400, height=1200, font=dict(size=17) )

fig.show()

In [36]:
#lexicalrichness barchart 
df_lex = df.groupby(df['Album']).mean()
df_lex =df_lex.reset_index()
fig = px.bar(df_lex, x='Album',y='lexical_richment')
fig.update_layout(barmode='group',title=dict(text="Lexicalrichness by album",y=0.99,x=0.5,xanchor='center',yanchor='top',font=dict(size=25)),
                  autosize=False, width=1400, height=1200, font=dict(size=17) )
fig.show()

In [37]:
#number of title by time line graph
df_group = df.groupby(['Artist','Year']).count().iloc[:,0:2]
df_group =df_group.reset_index()
fig = px.line(df_group, x='Year',y='Title')
fig.update_layout(title=dict(text="No. of Titles by Taylor Swift from 2000-2021",y=0.95,x=0.5,xanchor='center',yanchor='top',font=dict(size=25)))
fig.show()

In [38]:
#sentiment analyzer
def setimentanalyzer(df):
    neg='Negative'
    neu='Neutral'
    pos='Positive'
    negative = []
    neutral = []
    positive = []
    dominant_sentiment=[]
    dominant_sentiment_score=[]
    #Initialize the model
    sid = SentimentIntensityAnalyzer()
    #Iterate for each row of lyrics and append the scores
    for i in df.index:
       
        scores = sid.polarity_scores(df['Lyric'].iloc[i])
        negative.append(scores['neg'])
        neutral.append(scores['neu'])
        positive.append(scores['pos'])
        if scores['neg']>scores['pos']:
            dominant_sentiment_score.append(scores['neg'])
            dominant_sentiment.append(neg)
        elif scores['neg']<scores['pos']:
            dominant_sentiment_score.append(scores['pos'])
            dominant_sentiment.append(pos)
        else:
            dominant_sentiment_score.append(scores['neu'])
            dominant_sentiment.append(neu)
    #Create 5 columns to the main data frame  for each score
    df['negative'] = negative
    df['neutral'] = neutral
    df['positive'] = positive
    df['dominant_sentiment']=dominant_sentiment
    df['dominant_sentiment_score']=dominant_sentiment_score
    return df

In [39]:
#df sentiment
df_sentiment=setimentanalyzer(df)
df_sentiment = df_sentiment.drop(['Lyric', 'cleaned',
       'total_words', 'total_words_c', 'unique_words', 'unique_words_c',
       'lexical_richment'],axis=1)
print('df_sentiment shape: {}'.format(df_sentiment.shape))

df_sentiment shape: (363, 10)


In [40]:
#df positive
df_positive = df_sentiment.loc[df_sentiment.loc[:,'dominant_sentiment'] == 'Positive',:]
df_positive.sort_values(by=['positive'],ascending=False,inplace=True)
df_pos = df_positive.iloc[:20]
#df negative
df_negative = df_sentiment.loc[df_sentiment.loc[:,'dominant_sentiment'] == 'Negative',:]
df_negative.sort_values(by=['negative'],ascending=False,inplace=True)
df_neg = df_negative.iloc[:20]
#df neutral
df_neutral = df_sentiment.loc[df_sentiment.loc[:,'dominant_sentiment'] == 'Neutral',:]
df_neutral.sort_values(by=['neutral'],ascending=False,inplace=True)
df_neu = df_neutral.iloc[:20]

In [41]:
#top 20 positive sentiment songs
fig = px.bar(df_pos, x="Title",y="dominant_sentiment_score")
fig.update_layout(barmode='group',title=dict(text="Top 20 positive sentiment score songs from Taylor swift",y=0.97,x=0.5,xanchor='center',yanchor='top',font=dict(size=25)),
                  autosize=False, width=1150, height=800, font=dict(size=17) )
fig.show()

In [42]:
#top 20 negative sentiment songs
fig = px.bar(df_neg, x="Title",y="dominant_sentiment_score")
fig.update_layout(barmode='group',title=dict(text="Top 20 negative sentiment score songs from Taylor swift",y=0.97,x=0.5,xanchor='center',yanchor='top',font=dict(size=25)),
                  autosize=False, width=1050, height=800, font=dict(size=17) )
fig.show()

In [43]:
#dominant sentiment in songs 
fig = px.scatter(df_sentiment, x="dominant_sentiment", y="dominant_sentiment_score",color="dominant_sentiment")
fig.update_layout(title=dict(text="Scatter plot of dominant sentiment",y=0.97,x=0.45,xanchor='center',yanchor='top',font=dict(size=25)),
                  autosize=False, width=1200, height=800, font=dict(size=17) )
fig.show()

In [44]:
#sentiment by time
df_time = df_sentiment.sort_values(by=['Date'],ascending=True)
fig = px.line(df_time, x='Date',y='dominant_sentiment_score',color='dominant_sentiment')
fig.update_layout(barmode='group',title=dict(text="Sentiment of Taylor Swift songs by time",y=0.97,x=0.45,xanchor='center',yanchor='top',font=dict(size=25)),
                  autosize=False, width=1400, height=500, font=dict(size=17) )
fig.show()

In [ ]:
#uncleaned bar chart
data  = [0.37211666850418007, 0.4937262279795504, 0.5456832346447217, 0.31220945733240824, 0.8266875235549795, 0.505935286273305]
labels = ['Doc2Vec', 'USE', 'BERT', 'TFIDF', 'GloVE', 'FastText']
result = pd.DataFrame({'Cosine_similarity':data,'Methods':labels})
fig = px.bar(result, x="Methods",y="Cosine_similarity")
fig.update_layout(barmode='group',title=dict(text="",y=0.97,x=0.45,xanchor='center',yanchor='top',font=dict(size=25))
                  , autosize=False, width=1000, height=500, font=dict(size=17) )
fig.show()

In [ ]:
#cleaned bar chart
data = {"d2v": 0.39202946531022015, "USE": 0.151523647740684 ,"BERT": 0.49315902116359583, "TFIDF": 0.299595434220966, "GloVe": 0.8266875235549795}
result = pd.DataFrame.from_dict(data,orient='index')
result = result.reset_index()
result.columns= ['Methods','Cosine_similarity']
fig = px.bar(result, x="Methods",y="Cosine_similarity")
fig.update_layout(barmode='group',title=dict(text="",y=0.97,x=0.45,xanchor='center',yanchor='top',font=dict(size=25))
                  , autosize=False, width=1000, height=500, font=dict(size=17) )
fig.show()